In [1]:
from config import LLAMA_MODEL
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader,WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent,AgentExecutor
from langchain_community.chat_models.ollama import ChatOllama

import os
from langchain import hub
from langchain_ollama import OllamaLLM
# import langchainhub
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
os.environ["TAVILY_API_KEY"] = "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"

# embeddings = OllamaEmbeddings(model=LLAMA_MODEL)

# data = WebBaseLoader("https://docs.smith.langchain.com/overview")
# docs = data.load()

# print(docs)

# split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
# vector = FAISS.from_documents(documents=split,embedding=embeddings)

# vector.save_local('web_data')
#retreiver = vector.as_retriever()
retriever = FAISS.load_local('web_data',OllamaEmbeddings(model=LLAMA_MODEL),allow_dangerous_deserialization=True).as_retriever()

retriever_tool = create_retriever_tool(retriever,"langsmith","Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
# res=retriever.invoke("how to upload a dataset")

print("Data has been loaded")

tavily = TavilySearchResults()

tools = [retriever_tool,tavily]
#search = tavily.invoke("what is thefull form of ICC?")
#print(search)
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
prompt = hub.pull("hwchase17/react")
print(prompt)
# agent = create_tool_calling_agent(llm,tools,prompt)

# exec = AgentExecutor(agent=agent,tools=tools,verbose=True)
# val  = exec.invoke({"input": "hi!"})
# print(val)


Data has been loaded


c:\Users\umend\Documents\fastAPI\Langchain\venv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [6]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model=LLAMA_MODEL)

In [7]:
agent = create_tool_calling_agent(model,tools,prompt)

exec = AgentExecutor(agent=agent,tools=tools)

In [5]:
agent =model.bind_tools(tools=[tavily])
# agent.invoke('who is the chairman of ICC ?')

ValueError: 'llama3' did not respond with valid JSON. 
                Please try again. 
                Response: Here is my response:
```
{
  "tool": "tavily_search_results_json",
  "tool_input": {
    "query": "Chairman of ICC"
  }
}
```

In [6]:
from langchain.agents import create_react_agent
agent = create_react_agent(model,tools=tools,prompt=prompt)
exec = AgentExecutor(agent=agent,tools=tools,handle_parsing_errors=True)

In [8]:
exec

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{

In [8]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')